## Stroy 28. 정보은닉과 `__dict__`

### 속성 감추기

In [1]:
class Person:
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __str__(self):
        return f'{self.name}: {self.age}'

In [2]:
p = Person('Yoo', 21)
print(p)

p.age -= 1  # 프로그래머 실수 - 객체 외부에서 객체 내에 있는 변수(속성)에 직접 접근 허용
print(p)

Yoo: 21
Yoo: 20


---

In [3]:
class Person:
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def add_age(self, a):
        if a < 0:
            print('나이 정보 오류')
        else:
            self.age += a
        
    def __str__(self):
        return f'{self.name}: {self.age}'

In [4]:
p = Person('Yoo', 21)
print(p)

p.add_age(1)  # p.age -= 1 오류를 범할 가능성 여전히 있음
print(p)

Yoo: 21
Yoo: 22


---

In [5]:
class Person:
    
    def __init__(self, name, age):
        self.__name = name
        self.__age = age
        
    def add_age(self, a):
        if a < 0:
            print('나이 정보 오류')
        else:
            self.__age += a
        
    def __str__(self):
        return f'{self.__name}: {self.__age}'

In [6]:
p = Person('Yoo', 21)
print(p)

p.add_age(1)  # p.__age += 1 을 실행하면 오류 발생함
print(p)

Yoo: 21
Yoo: 22


 **약속**: 객체 내 변수(속성) 이름 앞에 언더바를 하나만 붙이면 이 변수에 접근하기 없기~~~~

In [7]:
class Person:
    
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    def add_age(self, a):
        if a < 0:
            print('나이 정보 오류')
        else:
            self._age += a
        
    def __str__(self):
        return f'{self._name}: {self._age}'

In [8]:
p = Person('Yoo', 21)
print(p)

p.add_age(1)  # p.__age += 1 이렇게 안 쓰기로 약속했다
print(p)

Yoo: 21
Yoo: 22


### `__dict__`

In [9]:
class Person:
    
    def __init__(self, name, age):
        self._name = name
        self._age = age

In [10]:
p = Person('Yoo', 21)
print(p.__dict__)

{'_name': 'Yoo', '_age': 21}


> 객체 내에는 `__dict__`이 있으며 딕셔너리이다

> `__dict__`에는 해당 객체의 변수 정보가 담긴다

In [11]:
p.len = 178
p.adr = 'Korea'

print(p.__dict__)

{'_name': 'Yoo', '_age': 21, 'len': 178, 'adr': 'Korea'}


In [12]:
p.__dict__['_name'] = 'James'
p.__dict__['_age'] += 1

print(p.__dict__)

{'_name': 'James', '_age': 22, 'len': 178, 'adr': 'Korea'}


> 객체 내에 있는 변수의 값은 사실 `__dict__`를 통해서 관리가 된다

In [13]:
class Person:
    
    def __init__(self, name, age):
        self.__name = name
        self.__age = age

In [14]:
p = Person('Yoo', 21)
print(p.__dict__)

{'_Person__name': 'Yoo', '_Person__age': 21}


> 변수 이름에 언더바를 두 개 붙이면 파이썬은 다음과 같은 패턴으로 이름을 바꾼다
  
> `__AttrName` -> `_ClassName__AttrName` 

## Story 29. `__slots__`의 효과

### `__dict__`의 단점과 그 해결책

In [15]:
class Point3D:
    
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
    def __str__(self):
        return f'({self.x}, {self.y}, {self.z})'

In [16]:
p1 = Point3D(1, 1, 1)
p2 = Point3D(24, 17, 31)

In [17]:
print(p1)
print(p2)

(1, 1, 1)
(24, 17, 31)


> Point3D가 수천 개이면, 수 천개의 딕셔너리가 필요해 시스템에 부담이 됨

In [18]:
def main():
    p = Point3D(1, 1, 1)
    
    for i in range(3000):
        for j in range(3000):
            p.x += 1
            p.y += 1
            p.z += 1

In [19]:
%timeit main()

3.36 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

In [20]:
class Point3D:
    
    __slots__ = ('x', 'y', 'z')  # 속성을 x, y, z로 제한한다
    
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
    def __str__(self):
        return f'({self.x}, {self.y}, {self.z})'

In [21]:
p1 = Point3D(1, 1, 1)
p2 = Point3D(24, 17, 31)

In [22]:
print(p1)
print(p2)

(1, 1, 1)
(24, 17, 31)


In [23]:
def main():
    p = Point3D(1, 1, 1)
    
    for i in range(3000):
        for j in range(3000):
            p.x += 1
            p.y += 1
            p.z += 1

In [24]:
%timeit main()

2.73 s ± 72.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
p1.w = 30  # w는 __slots__에 명시되어 있지 않은 이름이므로 오류!

AttributeError: 'Point3D' object has no attribute 'w'

## Story 30. 프로퍼티

### 안전하게 접근하기

In [26]:
class NaturalNumber:
    
    def __init__(self, n):        
        if n < 1:
            self.__n = 1
        else:
            self.__n = n
            
    def getn(self):  # 값 꺼내기, getter
        return self.__n
    
    def setn(self, n):  # 값 수정하기, setter        
        if n < 1:
            self.__n = 1
        else:
            self.__n = n       

In [27]:
n = NaturalNumber(-3)
print(n.getn())
n.setn(2)
print(n.getn())

1
2


---

In [28]:
class NaturalNumber:
    
    def __init__(self, n):     
        self.setn(n)

    def getn(self):  # 값 꺼내기, getter
        return self.__n
    
    def setn(self, n):  # 값 수정하기, setter       
        if n < 1:
            self.__n = 1
        else:
            self.__n = n     

In [29]:
n1 = NaturalNumber(1)
n2 = NaturalNumber(2)
n3 = NaturalNumber(3)

In [30]:
n1.setn(n2.getn() + n3.getn())  # 조금 복잡해 보인다 - n1.n = n2.n + n3.n ??

print(n1.getn())

5


---

In [31]:
class NaturalNumber:
    
    def __init__(self, n):     
        self.setn(n)

    def getn(self):  # 값 꺼내기, getter
        return self.__n
    
    def setn(self, n):  # 값 수정하기, setter       
        if n < 1:
            self.__n = 1
        else:
            self.__n = n
            
    n = property(getn, setn)  # 프로퍼티 설정

In [32]:
n1 = NaturalNumber(1)
n2 = NaturalNumber(2)
n3 = NaturalNumber(3)

In [33]:
n1.n = n2.n + n3.n

print(n1.n)

5


### property

In [34]:
class NaturalNumber:
    
    def __init__(self, n):     
        self.setn(n)
        
    n = property()  # property 객체 생성

    def getn(self):  # 값 꺼내기, getter
        return self.__n
    
    n = n.getter(getn)  # getn 메소드를 getter로 등록
    
    def setn(self, n):  # 값 수정하기, setter       
        if n < 1:
            self.__n = 1
        else:
            self.__n = n
            
    n = n.setter(setn)  # setn 메소드를 gsetter로 등록

In [35]:
n1 = NaturalNumber(1)
n2 = NaturalNumber(2)
n3 = NaturalNumber(3)

In [36]:
n1.n = n2.n + n3.n

print(n1.n)

5


---

In [37]:
class NaturalNumber:
    
    def __init__(self, n):     
        self.n = n
        
    n = property()  # property 객체 생성

    def pm(self):  # 값 꺼내기, getter
        return self.__n
    
    n = n.getter(pm)  # pm 메소드를 getter로 등록
    
    def pm(self, n):  # 값 수정하기, setter       
        if n < 1:
            self.__n = 1
        else:
            self.__n = n
            
    n = n.setter(pm)  # pm 메소드를 gsetter로 등록

### 또 다른 방식 - 더 많이 쓰임

In [38]:
class NaturalNumber:
    
    def __init__(self, n):     
        self.n = n
        
    @property
    def n(self): 
        return self.__n
    
    @n.setter    
    def n(self, n):   
        if n < 1:
            self.__n = 1
        else:
            self.__n = n

In [39]:
n1 = NaturalNumber(1)
n2 = NaturalNumber(2)
n3 = NaturalNumber(3)

In [40]:
n1.n = n2.n + n3.n

print(n1.n)

5


```python
@property
def n(self):
    return self.__n
```

> 이어서 등장하는 메소드 `n`을 getter로 지정하면서 `property` 객체 생성 그리고 이렇게 생성된 `property` 객체를 변수 `n`에 저장

```python
@n.setter
def n(self, n):
    # 메소드의 몸체 내용은 생략함
```

> 이어서 등장하는 메소드 `n`을 `n`으로 저장된 `property` 객체의 setter로 등록하고 이렇게 생성된 `property` 객체를 변수 `n`에 저장